<a href="https://colab.research.google.com/github/dolmani38/Summary2/blob/main/korean_sentence_construction_v0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# encoder/decoder를 이용하여 keyword를 입력하면, 유추할 수 있는 완성된 문장을 자동 생성한다.

# 여기서부터 테스트

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
#from torchtext.datasets import Multi30k
#from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
import urllib.request
import nltk
nltk.download('punkt')

# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n',' ')
    txt = txt.replace('\r',' ')    
    txt = txt.replace('=','')
    txt = txt.replace('\"','')   
    txt = txt.replace('\'','')
    #txt = txt.replace(',','')
    txt = txt.replace('..','')
    txt = txt.replace('...','')
    #txt = txt.replace('.','. ')
    txt = txt.replace('.','. ')
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')           
    txt = txt.replace('  ',' ')
    txt = txt.replace('  ',' ')    
    txt = txt.replace('  ',' ')   
    txt = txt.replace('  ',' ')             
    return txt.strip()

def collect_training_dataset_for_s2s_generator(source_urls=[]):
    ko_sentences_dataset = []
    for url in source_urls:
        raw_text = urllib.request.urlopen(url).read().decode('utf-8')
        ko_sentences_dataset += nltk.sent_tokenize(clean_text(raw_text))

    sentences = []
    for txt in ko_sentences_dataset:
        txt = txt.strip()
        if len(txt) > 40 and txt.endswith('다.'):
            #ko_grammar_dataset.append([txt,1])
            txt = txt.replace('.','')
            sentences.append(txt)

    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
if True:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


# dataset 구성

In [107]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/summary/korean_news_corpus.csv')
df

,title,contents
0,문 대통령 변창흠 국토장관 사의표명 사실상 수용,정만호 국민소통수석이 12일 오후 청와대 춘추관 대브리핑룸에서 변창흠 국토부 장관 ...
1,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
2,계급장 수여하는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
3,수상자 메달 걸어주는 문 대통령,(아산뉴스1) 이광호 기자 문재인 대통령이 12일 오후 충남 아산시 경찰대학에서 열...
4,정몽구 서울아산병원에 50억 쾌척,인재 양성·소외 계층 지원 등 계획 “부친 질병·가난 악순환 끊기 원해 국내 최고 ...
...,...,...
128815,스테인리스보다 1. 5배 단단…고엔트로피 합금기술 개발,- 포항공대 김형섭 교수팀 기존보다 3~10배 가격경쟁력 확보 헤테로구조 고엔트로피...
128816,철-구리 합금으로 더 강한 금속 소재 탄생,포스텍 다상 헤테로구조 고엔트로피 합금 개발 [아이뉴스24 최상국 기자] 서로 섞이...
128817,이에이트 한국기계연구원에 NFLOW SPH 공급,[파이낸셜뉴스]이에이트는 지난 6일 한국기계연구원(KIMM)에 유체 시뮬레이션 소프...
128818,이에이트 한국기계연구원에 전산유체역학(CFD) 솔루션 공급,이에이트 직원이 한국기계연구원(KIMM) 엔플로우 사용자를 대상으로 교육을 진행하고...


In [108]:
df = df.dropna(axis=0)
df['contents'].count()

128792

In [113]:
import re
import sys
# 검사...
pattens = ["[34569][0-9]{3}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}[\;.\;-\; ][0-9]{4}",
           "[0-9]{2,3}[\:\s\;.\;,\;-;)][0-9]{3,4}[\:\s\;.\;,\;-][0-9]{4}",
           "[0-9]{1}[0-9]{1}[\W]?[0-1]{1}[0-9]{1}[\W]?[0-3]{1}[\W]?[0-9]{1}[\W]?[1-4]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}[\W]?[0-9]{1}",
           "[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{3}[\:\s\;.\;,\;-]([0-9]{5,6}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{5}|[0-9]{2,3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{7}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{4,6}[\:\s\;.\;,\;-][0-9]|[0-9]{5}[\:\s\;.\;,\;-][0-9]{3}[\:\s\;.\;,\;-][0-9]{2}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{5}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{4}[\:\s\;.\;,\;-][0-9]{4}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{3}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{7})|[0-9]{4}[\:\s\;.\;,\;-]([0-9]{3}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{2}[\:\s\;.\;,\;-][0-9]{6}[\:\s\;.\;,\;-][0-9])|[0-9]{5}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{6}|[0-9]{6}[\:\s\;.\;,\;-][0-9]{2}[\:\s\;.\;,\;-][0-9]{5,6}"
           ]

filters = []
for p in pattens:
    filters.append(re.compile(p))

sentences = []
df = df.dropna(axis=0)
cnt = df['contents'].count()
#print('Total row count:',cnt)
i=0
for raw_text in df['contents']:
    i=i+1
    try:
        if i%100 == 0:
            percent = ("{0:.2f}").format(100 * (i / float(cnt)))
            print(f'\r {percent}% {i}/{str(cnt)}', end="", flush=True)

        docs = nltk.sent_tokenize(clean_text(raw_text))
        for txt in docs:
            if txt.find('▶') > -1 or txt.find('@') > -1 or txt.find('ⓒ') > -1: 
                pass
            else:
                txt = txt.strip()
                if any(chr.isdigit() for chr in txt) :
                    pass
                else:
                    sentences.append(txt)
    except KeyboardInterrupt as ki:
        raise ki        
    except:
        pass #print("Unexpected error:", sys.exc_info()[0])      



 99.93% 128700/128792

In [114]:
len(sentences)

2710820

In [115]:
import re
import sys
import io

#텍스트 정제(전처리)
def cleanText(readData):
    #텍스트에 포함되어 있는 특수 문자 제거
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》◆◇●🎧○▲\t―△━▷]', '', readData)
    return text

In [124]:
c_sentences = []
for sentence in sentences:
    s = cleanText(sentence)
    c = len(s.split())
    if c >= 3 and c < 10 and s.find('재배포') < 0 and s.find('기자') < 0  and s.find('유투브') < 0 and s.find('www') < 0 and s.find('com') < 0 and s.find('접속하기') < 0 and s.find('http') < 0 and s.find('뉴스') < 0 and s.find('일보') < 0 :
        if s.endswith(('다','요','까','죠','냐')):
            c_sentences.append(s.strip())

In [125]:
len(c_sentences)

849997

In [126]:
c_sentences[6000:]

['학력·경력과 무관하게 지원할 수 있다',
 '넥슨컴퍼니 채용 홈페이지에서 확인 가능하다',
 '눈길을 끄는 부분은 기존 정형화된 게임을 탈피한 엠오디MOD’와 페이스플레이’다',
 '엠오디MOD는 게임 제작 플랫폼’이다',
 '페이스플레이FACEPLAY도 눈길을 끈다',
 '신개념 놀이 플랫폼’으로 정의했다',
 '창의적인 도면을 제작해 다른 이용자와 공유하는 것도 가능하다',
 '티저 페이지 개설과 함께 게임 플레이 영상도 공개됐다',
 '국제 학술사회에서 평가는 이미 끝났다고 봐야 한다며 이같이 밝혔다',
 '호킹은 소변이 마려웠지만 그곳에는 장애인 화장실이 없었다',
 '이에 화가 머리끝까지 난 호킹은 충격적인 행동을 저지른다',
 '주방장이 놀라서 나와 소리를 지르자 호킹은 이렇게 답한다',
 '책은 대외적으로 알려지지 않았던 호킹의 인간적이고 내밀한 모습들을 보여준다',
 '물론 호킹의 과학적 성취들을 전하는 것도 저자는 잊지 않는다',
 '일견 비정해 보일 수 있으나 진실은 다르다',
 '학생들에 대한 애정이 남다른 호킹은 지원 요청을 항상 승인해줬다',
 '그의 비서 주디스는 이에 대해 이렇게 평한다',
 '정말 이상한 게임이죠',
 '이상한 사람들이 하는 이상한 게임이요',
 '서로를 얕봐서 그러는 건 아니에요',
 '가족 관계에 대한 호킹의 관점도 꽤나 진보적이었다',
 '요즘 진보파 사이에서 쓰이는 말로 그는 폴리아모리스트비독점적 다자연애주의자였다',
 '호킹의 첫 아내는 간병인이었던 제인 와일드 호킹이었다',
 '그때 일레인은 아직 데이비드 메이슨이라는 남자와 혼인 중인 상태였다',
 '이 기묘한 가족 관계는 한동안 지속됐다',
 '아직 여야가 합의를 보지 못하고 있습니다',
 '최진 대통령리더십연구원장과 짚어보겠습니다',
 '많이 부족하죠 미흡하죠',
 '박영선  더불어민주당 서울시장 후보  저 박영선 특검을 정식으로 건의합니다',
 '단호하게 책임을 물어야 합니다',
 '반드시 끊어내야 합니다',
 '앵커 박영선 후보가 특검카드를 꺼

In [12]:
!pip install keybert
!pip install sentence-transformers==0.3.0
!pip install transformers==3.0.2

     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 2.1MB 14.2MB/s 
     |████████████████████████████████| 1.2MB 52.3MB/s 
     |████████████████████████████████| 870kB 50.1MB/s 
     |████████████████████████████████| 3.3MB 50.9MB/s 
  Created wheel for keybert: filename=keybert-0.2.0-cp37-none-any.whl size=10599 sha256=a810321b6d78f41b67af0f8cd52019e1ee39e90210d672207524692302567a07
  Stored in directory: /root/.cache/pip/wheels/59/d7/16/04bab6677a4dfa9fd8ab2b350bac915d60f5378b83d6f5a372
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.4-cp37-none-any.whl size=114307 sha256=176b52a9e89be78c4880ae85c5e140053f021f66d6eef31e259ae762b5ef646e
  Stored in directory: /root/.cache/pip/wheels/26/ea/89/d0d2e013d951b6d23270aa9ca4018b82632ab7cd933c331316
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=6df7077fdf925403e6fa3ce54f2fd61e8e247b2ec3f2baef09d0eb220adde5a9
  Stored in 

## keyword 추출을 위한 keyBERT 준비

In [13]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("xlm-r-large-en-ko-nli-ststb")

100%|██████████| 1.80G/1.80G [01:33<00:00, 19.1MB/s]


In [14]:
key_model = KeyBERT(model=sentence_model)

In [15]:
def extract_key(sentence):
    #print(s)
    top_n= int(len(sentence.split())/3)
    keys = key_model.extract_keywords(sentence,top_n=top_n)
    x= {}
    for k,p in keys:
        x[k] = sentence.lower().find(k)
        #print(x[k],k)
    keys = list(dict(sorted(x.items(), key=lambda item: item[1])).keys())
    return keys




In [51]:
i = 200
[' '.join(extract_key(c_sentences[i])),c_sentences[i]]

['한강야생조류생태공원', '한폭의 그림 같은 한강야생조류생태공원 모습']

## 한국어 tokenizer 준비

In [17]:
# Copyright 2018 Google AI, Google Brain and Carnegie Mellon University Authors and the HuggingFace Inc. team and Jangwon Park
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Tokenization classes for KoBert model."""


import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            bos_token="[SOS]",
            eos_token="[EOS]",
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            bos_token=bos_token,
            eos_token=eos_token,
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

        self.token2idx[bos_token] = len(self.token2idx)
        self.idx2token.append(bos_token)
        self.token2idx[eos_token] = len(self.token2idx)
        self.idx2token.append(eos_token)   

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

    def tokenize_wl(self,sentence,max_length=128):
        tokens = ['[SOS]']+self.tokenize(sentence)+['[EOS]']
        '''
        if len(tokens) < max_length:
            tokens += ['[PAD]' for i in range(max_length-len(tokens))]
        else:
            tokens = tokens[0:max_length]
            tokens[max_length-1] = "[EOS]"
        '''    
        #print(len(tokens))
        #print(tokens)
        return tokens


In [18]:
pretraoned_kobert_model_name='monologg/kobert'
tokenizer = KoBertTokenizer.from_pretrained(pretraoned_kobert_model_name)

In [19]:
print(tokenizer.get_vocab())

{'[UNK]': 0, '[PAD]': 1, '[CLS]': 2, '[SEP]': 3, '[MASK]': 4, '!': 5, "!'": 6, '!”': 7, '"': 8, '#': 9, '$': 10, '%': 11, '%)': 12, '&': 13, '&#34;': 14, "'": 15, "'(": 16, "',": 17, '(': 18, '(0': 19, '(1': 20, '(10': 21, '(12': 22, '(15': 23, '(17': 24, '(18': 25, '(19': 26, '(2': 27, '(20': 28, '(23': 29, '(24': 30, '(25': 31, '(3': 32, '(4': 33, '(5': 34, '(6': 35, '(7': 36, '(8': 37, '(9': 38, '(?)': 39, ')': 40, ")'": 41, '),': 42, ')’': 43, '*': 44, '+': 45, ',': 46, '-': 47, '----------------': 48, '-1': 49, '-2': 50, '-20': 51, '-3': 52, '-4': 53, '.': 54, '...': 55, '..."': 56, "...'": 57, '...”': 58, '/': 59, '0': 60, '0%': 61, '0%)': 62, '0.0': 63, '0.00': 64, '0.1': 65, '0.1%': 66, '0.2%': 67, '0.3': 68, '0.3%': 69, '0.4%': 70, '0.5': 71, '0.5%': 72, '0.6': 73, '0.6%': 74, '0.7': 75, '0.7%': 76, '0.8': 77, '0.8%': 78, '00': 79, '000.0': 80, '00000': 81, '01': 82, '02': 83, '02-': 84, '03': 85, '04': 86, '05': 87, '06': 88, '07': 89, '08': 90, '09': 91, '0:00:00': 92, '1': 

In [20]:
# Print the sentence split into tokens.
toks = tokenizer.tokenize_wl(c_sentences[0])
print('Tokenized: ', toks)
# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(toks))

Tokenized:  ['[SOS]', '▁그동안', '▁팔', '아', '치', '웠', '던', '▁삼성전자', '와', '▁SK', '하이닉스', '▁등', '▁반도체', '주', '를', '▁다시', '▁담', '기', '▁시작했다', '[EOS]']
Token IDs:  [8002, 1193, 4814, 6797, 7483, 7036, 5842, 2653, 6983, 689, 7817, 1815, 2209, 7276, 6116, 1574, 1607, 5561, 2990, 8003]


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## dataset을 위한 Batch 구성

In [127]:
class Batch:
  src = []
  trg = []
max_length = 100
batch_size = 128 #4 # 64, 128
dataset_iterator = []
batch_counter = 0

sentences2 = c_sentences
for i in range(0,len(sentences2),batch_size):
#for i in range(0,1,batch_size):
    nSRC = []
    nTRG = []
    src_max = 0
    trg_max = 0
    batch = Batch()
    batch.src = []
    batch.trg = []
    
    percent = ("{0:.2f}").format(100 * (i / float(len(sentences2))))
    print(f'\r {percent}% {i}/{str(len(sentences2))}', end="", flush=True)
    for j in range(i,i+batch_size):
        try:
            sentence = sentences2[j]+'.'
            keys = extract_key(sentence)
            srct = tokenizer.convert_tokens_to_ids(tokenizer.tokenize_wl((' '.join(keys)).strip()))
            trgt = tokenizer.convert_tokens_to_ids(tokenizer.tokenize_wl(sentence))
            if len(srct) < max_length and len(trgt) < max_length:
                if len(trgt) > trg_max:
                    trg_max = len(trgt)            
                nTRG.append(trgt)
                if len(srct) > src_max:
                    src_max = len(srct)
                nSRC.append(srct)

        except Exception as e:
            #print('Error:',e)
            pass

    for s in nSRC:
        ss = s
        if len(s) < src_max:
            ss += [1 for i in range(src_max-len(s))]
        batch.src.append(ss)

    for t in nTRG:
        tt = t
        if len(t) < trg_max:        
            tt += [1 for i in range(trg_max-len(t))]
        batch.trg.append(tt)

    #print(len(batch.src),len(batch.src[0]))
    #print(len(batch.trg),len(batch.trg[0]))

    batch.src = torch.tensor(batch.src).to(device)
    batch.trg = torch.tensor(batch.trg).to(device)
    dataset_iterator.append(batch)


 25.71% 218496/849997

KeyboardInterrupt: ignored

In [128]:
len(dataset_iterator)

1707

In [129]:
dataset_iterator[1].src

tensor([[8002, 3016, 1076,  ...,    1,    1,    1],
        [8002, 3490, 6896,  ...,    1,    1,    1],
        [8002,  760,  517,  ...,    1,    1,    1],
        ...,
        [8002, 3341, 5550,  ...,    1,    1,    1],
        [8002, 1997, 6615,  ...,    1,    1,    1],
        [8002, 2136, 6983,  ...,    1,    1,    1]], device='cuda:0')

In [130]:
# dataset_iterator의 저장!

import pickle
with open('/content/drive/MyDrive/Colab Notebooks/summary/dataset_iterator.pkl', 'wb') as f:
    pickle.dump(dataset_iterator, f)

In [131]:
# dataset_iterator를 읽어들임.

import pickle
with open('/content/drive/MyDrive/Colab Notebooks/summary/dataset_iterator.pkl', 'rb') as f:
    dataset_iterator2 = pickle.load(f)

# encoder / decoder 구성

In [134]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [135]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [136]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [137]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [138]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [139]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

##seq2seq 구성

In [140]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

# Building the Model --> 실행

# Train 준비

In [141]:
INPUT_DIM = tokenizer.vocab_size
OUTPUT_DIM = tokenizer.vocab_size
#INPUT_DIM = len(tk.vocab)
#OUTPUT_DIM = len(tk.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [142]:
SRC_PAD_IDX = tokenizer.pad_token_id
TRG_PAD_IDX = tokenizer.pad_token_id
#SRC_PAD_IDX = tk.stoi['<pad>']
#TRG_PAD_IDX = tk.stoi['<pad>']
model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [143]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,159,940 trainable parameters


In [144]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights);

In [145]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [146]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        ##print(src.shape)
        ##print(trg.shape)
        output, _ = model(src, trg[:,:-1])
        ##print(output.shape)
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        ##print('loss',loss)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        ##print('epoch_loss',epoch_loss)
    return epoch_loss / len(iterator)

In [147]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [148]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Train !!!!

In [149]:
bcnt = len(dataset_iterator)
print(bcnt)
train_iterator = dataset_iterator[0:bcnt-int(bcnt/10)]
valid_iterator = dataset_iterator[bcnt-int(bcnt/10):bcnt]
#train_iterator = dataset_iterator[0:bcnt-1]
#valid_iterator = dataset_iterator[bcnt-1:bcnt]
print('train',len(train_iterator))
print('valid',len(valid_iterator))

1707
train 1537
valid 170


In [150]:
train_iterator[0].src

tensor([[8002, 4814, 6797,  ..., 8003,    1,    1],
        [8002, 2915, 7872,  ...,    1,    1,    1],
        [8002, 2546, 7996,  ...,    1,    1,    1],
        ...,
        [8002, 1986, 5377,  ...,    1,    1,    1],
        [8002, 1562, 6113,  ...,    1,    1,    1],
        [8002, 2756, 5468,  ...,    1,    1,    1]], device='cuda:0')

In [ ]:

N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/summary/tut6-model.pt')
        print('Save model!')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Save model!
Epoch: 01 | Time: 1m 56s
	Train Loss: 3.579 | Train PPL:  35.820
	 Val. Loss: 3.537 |  Val. PPL:  34.355
Save model!
Epoch: 02 | Time: 1m 57s
	Train Loss: 2.875 | Train PPL:  17.725
	 Val. Loss: 3.214 |  Val. PPL:  24.877
Save model!
Epoch: 03 | Time: 1m 57s
	Train Loss: 2.493 | Train PPL:  12.099
	 Val. Loss: 3.017 |  Val. PPL:  20.434
Save model!
Epoch: 04 | Time: 1m 57s
	Train Loss: 2.259 | Train PPL:   9.577
	 Val. Loss: 2.918 |  Val. PPL:  18.497
Save model!
Epoch: 05 | Time: 1m 57s
	Train Loss: 2.103 | Train PPL:   8.189
	 Val. Loss: 2.861 |  Val. PPL:  17.472
Save model!
Epoch: 06 | Time: 1m 57s
	Train Loss: 1.988 | Train PPL:   7.299
	 Val. Loss: 2.820 |  Val. PPL:  16.778
Save model!
Epoch: 07 | Time: 1m 57s
	Train Loss: 1.896 | Train PPL:   6.661
	 Val. Loss: 2.793 |  Val. PPL:  16.324
Save model!
Epoch: 08 | Time: 1m 57s
	Train Loss: 1.822 | Train PPL:   6.187
	 Val. Loss: 2.773 |  Val. PPL:  16.010
Save model!
Epoch: 09 | Time: 1m 57s
	Train Loss: 1.762 | Train 

In [ ]:
def complete_sentence(sentence, model, device, max_len = 50):
    
    model.eval()
    '''
    if isinstance(sentence, str):
        nlp = spacy.load('de_core_news_sm')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
    '''
    tokens = tokenizer.tokenize_wl(sentence)
    tokens = [tokenizer.bos_token] + tokens + [tokenizer.eos_token]
    

    src_indexes = [tokenizer.convert_tokens_to_ids(token) for token in tokens]
    #src_indexes = get_stoi(tk,sentence)
    print(src_indexes)
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    print(src_tensor)
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [tokenizer.bos_token_id]
    

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == tokenizer.eos_token_id: #tk.stoi['<eos>']: #.eos_token_id: # trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = tokenizer.convert_ids_to_tokens(trg_indexes) # :[tk.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [ ]:
trg, _ = complete_sentence('북한 미국 미사일',model,device)

In [ ]:
print(''.join(trg))

In [ ]:
extract_key(sentences[101])

In [ ]:
trg, _ = complete_sentence('차갑게 공주의 일어서려던 그라시그',model,device)
print(' '.join(trg))